In [1]:
from datasets import load_from_disk, concatenate_datasets, Dataset
import os
import json
import pandas as pd

datasets_dirs = [
    '/mnt/sea/speech/processed_datasets/Common_Voice_16_1_pa-IN_ASR',
    '/mnt/sea/speech/processed_datasets/fleurs_pa_ASR',
    '/mnt/sea/speech/ds_filtered/indicsuperb_pa_ASR_filtered',
    '/mnt/sea/speech/processed_datasets/Indicvoice_pa_ASR',
    '/mnt/sea/speech/ds_filtered/shrutilipi_pa_ASR_filtered' 
]

# cosmopedia_shards = ['/mnt/sea/it2_translation_shards/stories/translated_0', '/mnt/sea/it2_translation_shards/web_samples_v1/translated_0', '/mnt/sea/it2_translation_shards/web_samples_v2/translated_0']

indictts_manifest_json = '/mnt/sea/speech/indictts_ds/punjabi/manifest.json'
wikipedia = '/mnt/sea/speech/kenlm_wikipedia/train-00000-of-00001.parquet'

wiki_ds = Dataset.from_parquet(wikipedia)


df = pd.read_json(f'{indictts_manifest_json}', lines=True)
indic_ds = Dataset.from_pandas(df)

dss = []
# for d in cosmopedia_shards:
#     try:
#         ds = load_from_disk(f'{d}')
#         dss.append(ds)
#     except Exception as e:
#         print(d)

all_data_splits = []
for d in datasets_dirs:
    ds = load_from_disk(f'{d}')
    for split in ds:
        all_data_splits.append(ds[split])



/home/kd/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(all_data_splits)
print(indic_ds)
print(wiki_ds)
print(dss)



[Dataset({
    features: ['speaker_id', 'audio', 'text', 'gender', 'duration'],
    num_rows: 166
}), Dataset({
    features: ['speaker_id', 'audio', 'text', 'gender', 'duration'],
    num_rows: 487
}), Dataset({
    features: ['speaker_id', 'audio', 'text', 'gender', 'duration'],
    num_rows: 286
}), Dataset({
    features: ['audio', 'text', 'gender', 'duration', 'speaker_id'],
    num_rows: 1923
}), Dataset({
    features: ['audio', 'text', 'gender', 'duration', 'speaker_id'],
    num_rows: 251
}), Dataset({
    features: ['audio', 'text', 'gender', 'duration', 'speaker_id'],
    num_rows: 574
}), Dataset({
    features: ['audio', 'text', 'speaker_id', 'gender', 'duration'],
    num_rows: 22268
}), Dataset({
    features: ['audio', 'text', 'speaker_id', 'gender', 'duration'],
    num_rows: 7468
}), Dataset({
    features: ['audio', 'text', 'speaker_id', 'gender', 'duration'],
    num_rows: 17300
}), Dataset({
    features: ['audio', 'text', 'speaker_id', 'gender', 'duration'],
    n

In [3]:
from tqdm import tqdm
# get text from all dataset splits and create a list
all_texts_list = []
for d in tqdm(dss):
    all_texts_list.extend(d['translated'])
print(len(all_texts_list))
all_texts_list.extend(wiki_ds['text'])
print(len(all_texts_list))
all_texts_list.extend(indic_ds['text'])
print(len(all_texts_list))
for d in all_data_splits:
    all_texts_list.extend(d['text'])
print(len(all_texts_list))

0it [00:00, ?it/s]

0


51423
65950
145918


In [4]:
# config params
base_dir = '/home/kd/Desktop/proj/apr/speech_pa/'
vocab_has_vowels = True
convert_num_to_words = False
tokenizer_path = './'
ds_txt_file_name = f"{base_dir}kenlm_pa_ds_3.txt"
kenlm_train_txt_file = f'{base_dir}kenlm_pa_ds_3.txt'
kenlm_output_arpa_file = f'{base_dir}5gram_pa_3.arpa'
kenlm_output_correct_arpa_file = kenlm_output_arpa_file.replace('.arpa', '_correct.arpa')
kenlm_output_correct_bin_file = kenlm_output_correct_arpa_file.replace('.arpa', '.bin')
processor_with_lm_save_path = f"{base_dir}/wav2vec2-bert-pa-lm-processor-all_3"

import speech_utils as su 
from transformers import Wav2Vec2CTCTokenizer
import re

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(tokenizer_path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

vocab_dict = tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}
chars = sorted_vocab_dict.keys()
remove_chars = ['|', '[unk]', '[pad]', '<s>', '</s>']
chars = [c for c in chars if c not in remove_chars]
chars.append(' ')
print(chars)

# Required if we trained the wav2vec-bert with no vowels vocab
replacements = {
    'ਆ': 'ਅਾ',
    'ਇ': 'ਿੲ',
    'ਈ': 'ੲੀ',
    'ਉ': 'ੳੁ',
    'ਊ' :'ੳੂ',
    'ਏ': 'ੲੇ',
    'ਐ': 'ਅੈ',
    'ਔ': 'ਅੌ',
}
texts = []
for text in tqdm(all_texts_list):
    if convert_num_to_words:
        text = su.num_to_words(text)

    if not vocab_has_vowels:
        for k, v in replacements.items():
            text = text.replace(k, v)

    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    texts.append(text)


/home/kd/Desktop/proj/apr/speech_pa/speech_utils.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/home/kd/anaconda3/envs/bharat/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/kd/anaconda3/envs/bharat/lib/python3.10/site-packages/transformers/utils/generic.py:485: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use 

['ਂ', 'ਅ', 'ਆ', 'ਇ', 'ਈ', 'ਉ', 'ਊ', 'ਏ', 'ਐ', 'ਓ', 'ਔ', 'ਕ', 'ਖ', 'ਗ', 'ਘ', 'ਙ', 'ਚ', 'ਛ', 'ਜ', 'ਝ', 'ਞ', 'ਟ', 'ਠ', 'ਡ', 'ਢ', 'ਣ', 'ਤ', 'ਥ', 'ਦ', 'ਧ', 'ਨ', 'ਪ', 'ਫ', 'ਬ', 'ਭ', 'ਮ', 'ਯ', 'ਰ', 'ਲ', 'ਲ਼', 'ਵ', 'ਸ਼', 'ਸ', 'ਹ', '਼', 'ਾ', 'ਿ', 'ੀ', 'ੁ', 'ੂ', 'ੇ', 'ੈ', 'ੋ', 'ੌ', '੍', 'ਖ਼', 'ਗ਼', 'ਜ਼', 'ੜ', 'ਫ਼', 'ੰ', 'ੱ', 'ੲ', 'ੳ', ' ']


100%|██████████| 145918/145918 [00:08<00:00, 17622.86it/s] 


In [5]:
with open(ds_txt_file_name, "w", buffering=8192) as file:
    for text in texts:
        file.write(text + " ")

In [6]:
texts[-1]

'ਮੰਤਰੀ ਨੇ ਕਿਹਾ ਕਿ ਦਿੱਲੀ ਮੁੰਬਈ ਮੁੱਖ ਮਾਰਗ ਤੇ ਉਨ੍ਹਾਂ ਦਾ ਮੰਤਰਾਲਾ ਬਿਜਲੀ ਨਾਲ ਚੱਲਣ ਵਾਲੇ ਵਾਹਨਾਂ ਲਈ ਬੁਨਿਆਦੀ ਢਾਂਚੇ ਨੂੰ ਉਤਸ਼ਾਹਿਤ ਕਰਨ ਲਈ ਚਾਰਜਿੰਗ ਸਟੇਸ਼ਨ ਐਲ ਐਨ ਜੀ ਸਟੇਸ਼ਨਾਂ ਤੇ ਪੂੰਜੀ ਨਿਵੇਸ਼ ਕਰੇਗਾ'

In [9]:
!/mnt/sea/proj/apr/kenlm/build/bin/lmplz -o 5 < {kenlm_train_txt_file} > {kenlm_output_arpa_file} -S 90% --skip_symbols

=== 1/5 Counting and sorting n-grams ===
Reading /home/kd/Desktop/proj/apr/speech_pa/kenlm_pa_ds_3.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 18221306 types 802665
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:9631980 2:5898565120 3:11059810304 4:17695694848 5:25806223360
Statistics:
1 802664 D1=0.75451 D2=1.04411 D3+=1.26913
2 5317881 D1=0.815085 D2=1.08912 D3+=1.29471
3 11451226 D1=0.889659 D2=1.1995 D3+=1.34996
4 15044514 D1=0.944361 D2=1.31942 D3+=1.39561
5 16559530 D1=0.916065 D2=1.67761 D3+=1.39807
Memory estimate for binary LM:
type      MB
probing 1032 assuming -p 1.5
probing 1217 assuming -r models -p 1.5
trie     524 without quantization
trie     301 assuming -q 8 -b 8 quantization 
trie     461 assuming -a 22 array pointer compression
trie     238 assuming -a 22 -q 8 -b 8 array p

In [10]:
with open(kenlm_output_arpa_file, "r") as read_file, open(kenlm_output_correct_arpa_file, "w") as write_file:
  has_added_eos = False
  for line in  read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)


In [ ]:
!/mnt/sea/proj/apr/kenlm/build/bin/build_binary {kenlm_output_correct_arpa_file} {kenlm_output_correct_bin_file}

In [12]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import SeamlessM4TFeatureExtractor
from transformers import Wav2Vec2BertProcessor
from transformers import Wav2Vec2BertForCTC

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(tokenizer_path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = SeamlessM4TFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0")
processor = Wav2Vec2BertProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)

vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path=kenlm_output_correct_bin_file,
)

from m4t_processor_with_lm import M4TProcessorWithLM

processor_with_lm = M4TProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

processor_with_lm.save_pretrained(processor_with_lm_save_path)



/home/kd/anaconda3/envs/bharat/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading the LM will be faster if you build a binary file.
Reading /home/kd/Desktop/proj/apr/speech_pa/5gram_pa_3_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.
